## Webscrapper Jobs: Monster

In [1]:
import requests
from bs4 import BeautifulSoup

job_type = "data"
loc = "San Antonio, TX"

URL = f"https://www.monster.com/jobs/search/?q={job_type}\
        &where= {loc}"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="ResultsContainer")

# Look for Python jobs
python_jobs = results.find_all("h2", string=lambda t: "python" in t.lower())
for p_job in python_jobs:
    link = p_job.find("a")["href"]
    print(p_job.text.strip())
    print(f"Apply here: {link}\n")

# Print out all available jobs from the scraped webpage
job_elems = results.find_all("section", class_="card-content")
for job_elem in job_elems:
    title_elem = job_elem.find("h2", class_="title")
    company_elem = job_elem.find("div", class_="company")
    location_elem = job_elem.find("div", class_="location")
    if None in (title_elem, company_elem, location_elem):
        continue
    print(title_elem.text.strip())
    print(company_elem.text.strip())
    print(location_elem.text.strip())
    print()

Nurse, TBI Clinical Data Specialist
Vista Defense Technologies, LLC
Fort Sam Houston, TX

Data Engineer
WinCorp Solutions
San Antonio, TX

Data Scientist
Alaka`ina Foundation Family of Companies
San Antonio, TX

Data Engineer
Apex Systems
San Antonio, TX

LEAD COMPUTER SCIENTIST - SR. COMPUTER SCIENTIST - SPACE DATA SYSTEMS
Southwest Research Institute
San Antonio, TX

Sr Data Entry Operator
Randstad
San Antonio, TX

Sr. Data Engineer
Randstad Technologies
San Antonio, TX

Data Entry Specialist
Tri-Starr Personnel
San Antonio, TX

Data Control Clerk
System One
San Antonio, TX

Social Media Specialist II
CHRISTUS Health
San Antonio, TX

Aircraft Sheetmetal Mechanic
Apollo Professional Solutions
San Antonio, CA

Veterinary Technician - 000210
Banfield Pet Hospital
San Antonio, TX

Junior .Net Developer
CyberCoders
San Antonio, TX

Contact Center Trainer
OKIN BPS
San Antonio, TX

Bookkeeper
Ramstin III LTD
SAN ANTONIO, TX

Operations Manager
Phyllis Browning Company
San Antonio, TX

Marke

## Webscrapper Jobs: Seven Days

In [7]:
import requests
from bs4 import BeautifulSoup

emp_type = "education"
loc = "any"

URL = f"https://jobs.sevendaysvt.com/browse-jobs/?search_keywords={emp_type}&search_location={any}#.X0UFWElOnOQ"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="container")

# Look for jobs
jobs = results.find_all("h2", string=lambda t: "data" in t.lower())
for job in jobs:
    link = job.find("a")["href"]
    print(job.text.strip())
    print(f"Apply here: {link}\n")

# Print out all available jobs from the scraped webpage
job_elems = results.find_all("section", class_="card-content")
for job_elem in job_elems:
    title_elem = job_elem.find("h2", class_="title")
    company_elem = job_elem.find("div", class_="company")
    location_elem = job_elem.find("div", class_="location")
    if None in (title_elem, company_elem, location_elem):
        continue
    print(title_elem.text.strip())
    print(company_elem.text.strip())
    print(location_elem.text.strip())
    print()

AttributeError: 'NoneType' object has no attribute 'find_all'